# 汽车租赁案例 2

## 目标和前提条件

在本示例中(《汽车租赁优化 I》的扩展版),您将了解如何使用数学优化来确定汽车租赁公司应该在哪些地点扩大维修能力。我们将指导您使用 Gurobi Python API 创建此问题的混合整数线性规划模型,然后向您展示如何使用 Gurobi 求解器找到最优解。

此模型是 H. Paul Williams 所著《数学规划中的模型构建》第五版中的示例 26,见第 287 页和第 342-343 页。

这是一个中级示例,我们假设您了解 Python 和 Gurobi Python API,并且对构建数学优化模型有一定了解。

**下载代码库** <br /> 
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

---
## 问题描述

一家小型汽车租赁公司(只租赁一种类型的汽车)在格拉斯哥、曼彻斯特、伯明翰和普利茅斯设有租车点。对于每个工作日(周日公司休息)都有估计的需求量。这些估计数据如下表所示。不需要满足所有需求。

![weeklyDemand](weeklyDemand.PNG)


汽车可以租用一天、两天或三天,并可以在第二天早上归还到原租车点或其他租车点。例如,周四租用2天意味着汽车必须在周六早上归还;周五租用3天意味着汽车必须在下周二早上归还。周六租用1天意味着汽车必须在周一早上归还。

租期与起始地和目的地无关。根据历史数据,公司知道租期的分布情况:55%的汽车租用一天,20%租用两天,25%租用三天。根据当前估计,从各租车点租出并归还到给定租车点的汽车百分比(与日期无关)如下表所示。

![FromToPct](FromToPct.PNG)

公司租出一辆汽车的边际成本(包括'磨损'、管理等)估计如下:

| 租期 | 边际成本 |
| --- | --- |
| 1天 | $\$ 20$ |
| 2天 | $\$ 25$ |
| 3天 | $\$ 30$ |

拥有一辆汽车的'机会成本'(资本利息、存储、维护等)为每周$\$ 15$。

未损坏的汽车可以在租车点之间转移,不受距离限制。在转移日期内汽车不能租出。每辆汽车的转移成本(美元)如下表所示。

![FromToCst](FromToCst.PNG)

客户归还的汽车中有10%会损坏。发生这种情况时,客户需支付$\$ 100$的额外费用(无论损坏程度如何,公司都完全由保险覆盖)。此外,汽车必须转移到维修点,在第二天进行维修。
损坏汽车的转移成本与未损坏汽车相同(除非维修点就是当前租车点,
那么成本为$\$0$)。
损坏汽车的转移需要一天时间,除非它已经在维修点。到达维修点后,所有类型的维修(或更换)都需要一天时间。
只有两个租车点有维修能力。每个维修点的日维修能力(辆/天)如下:

| 维修点 | 维修能力 |
| --- | --- |
| 曼彻斯特 | 12 |
| 伯明翰 | 20 |

维修完成后,汽车第二天即可在维修点出租,或者可以转移到其他租车点(需要一天时间)。例如,在周三早上归还的损坏汽车如果需要转移到维修点(当前租车点除外),会在周三进行转移,周四维修,周五早上可在维修点出租。

租金取决于租期以及是否归还到同一租车点。价格(美元)如下表所示。

![RentalPrice](RentalPrice.PNG)

我们假设每天早上按以下顺序进行:
1. 客户归还到期的汽车。
2. 损坏的汽车被送到维修点。
3. 从其他租车点转移来的汽车到达。
4. 需要转移的汽车开始转移。
5. 汽车被租出。
6. 如果是维修点,那么修好的汽车可以出租。

汽车租赁公司想要确定在哪些地点扩大维修能力。每周固定成本(包括扩张所需贷款的利息支付)如下所示。选项包括:

1. 在伯明翰扩大维修能力,每天增加5辆车,每周固定成本为$\$18,000$。
2. 在伯明翰进一步扩大维修能力,每天再增加5辆车,每周固定成本为$\$8,000$。
3. 在曼彻斯特扩大维修能力,每天增加5辆车,每周固定成本为$\$20,000$。
4. 在曼彻斯特进一步扩大维修能力,每天再增加5辆车,每周固定成本为$\$5,000$。
5. 在普利茅斯建立维修能力,每天可维修5辆车,每周固定成本为$\$19,000$。

如果选择任何选项,必须完全执行;也就是说,不能进行部分扩张。另外,租车点的进一步扩张只有在第一次扩张也执行的情况下才能进行 - 例如,只有在选择了选项(1)的情况下才能选择伯明翰的选项(2)。如果选择了选项(2),那么也必须选择选项(1),这算作两个选项。曼彻斯特的扩张也有类似的规定。最多只能执行三个选项。

目标是确定在哪些地点扩大维修能力,汽车租赁公司应该拥有多少辆车,以及每周工作日开始时应该将它们放在哪个租车点,以最大化每周利润。公司希望找到一个'稳态'解决方案,使得每周相同的工作日在相同的租车点都有相同的预期汽车数量。

---
## 模型公式

$d,d2 \in \text{Depots}=\{\text{格拉斯哥}, \text{曼彻斯特}, \text{伯明翰},  \text{普利茅斯}\}$

$\text{NRD}=\{\text{格拉斯哥}\}$: 无维修能力的租车点

$\text{RD}=\{\text{曼彻斯特}, \text{伯明翰}, \text{普利茅斯}\}$: 有维修能力的租车点

$t \in \text{Days}=\{\text{周一},\text{周二},\text{周三},\text{周四},\text{周五},\text{周六}\}$

$r \in \text{RentDays}=\{1,2,3\}$: 租期天数

### 参数

$\text{demand}_{d,t} \in \mathbb{R}^+$: 租车点 $d$ 在 $t$ 日的估计租车需求

$\text{pctDepot}_{d,d2} \in \mathbb{R}^+$: 从租车点 $d$ 租出并归还到租车点 $d2$ 的汽车比例

$\text{cstTransfer}_{d,d2} \in \mathbb{R}^+$: 从租车点 $d$ 转移到租车点 $d2$ 的成本

$\text{pctRent}_{r} \in \mathbb{R}^+$: 租期为 $r$ 天的汽车比例

$\text{capRepair}_{d} \in \mathbb{R}^+$: 租车点 $d$ 的维修能力

$\text{cstSameDepot}_{r} \in \mathbb{R}^+$: 租期 $r$ 天且归还到同一租车点的租金

$\text{cstOtherDepot}_{r} \in \mathbb{R}^+$: 租期 $r$ 天且归还到其他租车点的租金

$\text{marginalCost}_{r} \in \mathbb{R}^+$: 公司租出一辆汽车 $r$ 天的边际成本

$\text{pctUndamaged } \in [0,1]$: 客户归还未损坏汽车的比例

$\text{pctDamaged  } \in [0,1]$: 客户归还损坏汽车的比例

$\text{cstOwn} \in \mathbb{R}^+$: 拥有一辆汽车的成本

$\text{damagedFee} = 10$: 损坏汽车费用。10%的汽车会损坏,每辆损坏车收取$\$100$。

$\text{cstExpCapB1} \in \mathbb{R}^+$: 伯明翰扩大维修能力的每周固定成本

$\text{cstExpCapB2} \in \mathbb{R}^+$: 伯明翰进一步扩大维修能力的每周固定成本

$\text{cstExpCapM1} \in \mathbb{R}^+$: 曼彻斯特扩大维修能力的每周固定成本

$\text{cstExpCapM2} \in \mathbb{R}^+$: 曼彻斯特进一步扩大维修能力的每周固定成本

$\text{cstExpCapP} \in \mathbb{R}^+$: 普利茅斯扩大维修能力的每周固定成本

### 决策变量

$\text{xOwned} \in \mathbb{R}^+$: 拥有的汽车总数

$\text{xUndamaged}_{d,t} \in \mathbb{R}^+$: 在 $t$ 日开始时租车点 $d$ 可用的未损坏汽车数量

$\text{xDamaged}_{d,t} \in \mathbb{R}^+$: 在 $t$ 日开始时租车点 $d$ 可用的损坏汽车数量

$\text{xRented}_{d,t} \in \mathbb{R}^+$: 在 $t$ 日开始时从租车点 $d$ 租出的汽车数量

$\text{xUDleft}_{d,t} \in \mathbb{R}^+$: 在 $t$ 日结束时租车点 $d$ 剩余的未损坏汽车数量

$\text{xDleft}_{d,t} \in \mathbb{R}^+$: 在 $t$ 日结束时租车点 $d$ 剩余的损坏汽车数量

$\text{xUDtransfer}_{d,d2,t} \in \mathbb{R}^+$: 在 $t$ 日开始时从租车点 $d$ 转移到租车点 $d2$ 的未损坏汽车数量

$\text{xDtransfer}_{d,d2,t} \in \mathbb{R}^+$: 在 $t$ 日开始时从租车点 $d$ 转移到租车点 $d2$ 的损坏汽车数量

$\text{xRepaired}_{d,t} \in \mathbb{R}^+$: 在 $t$ 日期间在租车点 $d$ 维修的损坏汽车数量

$\text{yExpandCapB1} \in \{0,1 \}$: 如果伯明翰的维修能力扩大了每天5辆车,则该二元变量等于1

$\text{yExpandCapB2} \in \{0,1 \}$: 如果伯明翰的维修能力进一步扩大了每天5辆车,则该二元变量等于1

$\text{yExpandCapM1} \in \{0,1 \}$: 如果曼彻斯特的维修能力扩大了每天5辆车,则该二元变量等于1

$\text{yExpandCapM2} \in \{0,1 \}$: 如果曼彻斯特的维修能力进一步扩大了每天5辆车,则该二元变量等于1

$\text{yExpandCapP} \in \{0,1 \}$: 如果普利茅斯的维修能力扩大了每天5辆车,则该二元变量等于1

### 目标函数
目标是最大化利润。

\begin{equation}
\sum_{d \in \text{Depots}}
\sum_{t \in \text{Days}}
\sum_{r \in \text{RentDays}} 
\text{pctDepot}_{d,d}*\text{pctRent}_{r}*(\text{cstSameDepot}_{r} - \text{marginalCost}_{r} + \text{damagedFee})*\text{xRented}_{d,t}
\end{equation}

\begin{equation}
+ \sum_{d \in \text{Depots}} \sum_{d2 \in \text{Depots}}
\sum_{t \in \text{Days}}
\sum_{r \in \text{RentDays}} 
\text{pctDepot}_{d,d2}*\text{pctRent}_{r}*(\text{cstOtherDepot}_{r} - \text{marginalCost}_{r} + \text{damagedFee})*\text{xRented}_{d,t}
\end{equation}

\begin{equation}
- \sum_{d \in \text{Depots}} \sum_{d2 \in \text{Depots}}
\sum_{t \in \text{Days}} \text{cstTransfer}_{d,d2}*(\text{xUDtransfer}_{d,d2,t} + \text{xDtransfer}_{d,d2,t} )
- \text{cstOwn}*\text{xOwned}
\end{equation}

\begin{equation}
-(\text{cstExpCapB1}*\text{yExpandCapB1} + \text{cstExpCapB2}*\text{yExpandCapB2} + \text{cstExpCapM1}*\text{yExpandCapM1} 
\end{equation}

\begin{equation}
+ \text{cstExpCapM2}*\text{yExpandCapM2} + \text{cstExpCapP}*\text{yExpandCapP} )
\end{equation}

### 约束条件

**无维修能力租车点的未损坏汽车** <br />
在 $t$ 日开始时无维修能力租车点 $d$ 可用的未损坏汽车数量。

\begin{equation}
\sum_{d2 \in \text{Depots}} 
\sum_{r \in \text{RentDays}} \text{pctUndamaged}*\text{pctDepot}_{d2,d}*\text{pctRent}_{r}*\text{xRented}_{d2,(t-r)mod(6)}
\end{equation}

\begin{equation}
+ \sum_{d2 \in \text{Depots}} \text{xUDtransfer}_{d2,d,(t-1)mod(6)} + \text{xUDleft}_{d,(t-1)mod(6)} = \text{xUndamaged}_{d,t} 
\quad \forall d \in NRD, t \in Days
\end{equation}

在 $t$ 日期间无维修能力租车点 $d$ 的未损坏汽车需求。

\begin{equation}
\text{xUndamaged}_{d,t} = \text{xRented}_{d,t} + 
\sum_{d2 \in \text{Depots}} \text{xUDtransfer}_{d,d2,t} + \text{xUDleft}_{d,t}
\quad \forall d \in NRD, t \in Days
\end{equation}

**有维修能力租车点的未损坏汽车** <br />
在 $t$ 日开始时有维修能力租车点 $d$ 可用的未损坏汽车数量。

\begin{equation}
\sum_{d2 \in \text{Depots}} 
\sum_{r \in \text{RentDays}} \text{pctUndamaged}*\text{pctDepot}_{d2,d}*\text{pctRent}_{r}*\text{xRented}_{d2,(t-r)mod(6)}
\end{equation}

\begin{equation}
+ \sum_{d2 \in \text{Depots}} \text{xUDtransfer}_{d2,d,(t-1)mod(6)} 
+ \text{xRepaired}_{d,(t-1)mod(6)}  + \text{xUDleft}_{d,(t-1)mod(6)} = \text{xUndamaged}_{d,t} 
\quad \forall d \in NRD, t \in Days
\end{equation}

在 $t$ 日期间有维修能力租车点 $d$ 的未损坏汽车需求。

\begin{equation}
\text{xUndamaged}_{d,t} = \text{xRented}_{d,t} + 
\sum_{d2 \in \text{Depots}} \text{xUDtransfer}_{d,d2,t} + \text{xUDleft}_{d,t}
\quad \forall d \in NRD, t \in Days
\end{equation}

**无维修能力租车点的损坏汽车** <br />
在 $t$ 日开始时无维修能力租车点 $d$ 可用的损坏汽车数量。

\begin{equation}
\sum_{d2 \in \text{Depots}} 
\sum_{r \in \text{RentDays}} \text{pctDamaged}*\text{pctDepot}_{d2,d}*\text{pctRent}_{r}*\text{xRented}_{d2,(t-r)mod(6)}
\end{equation}

\begin{equation}
+ \text{xDleft}_{d,(t-1)mod(6)} = \text{xDamaged}_{d,t} \quad \forall d \in NRD, t \in Days
\end{equation}

在 $t$ 日期间无维修能力租车点 $d$ 的未损坏汽车需求。

\begin{equation}
\text{xDamaged}_{d,t} = 
\sum_{d2 \in \text{Depots} \cap RD} \text{xDtransfer}_{d,d2,t} + \text{xDleft}_{d,t}
\quad \forall d \in NRD, t \in Days
\end{equation}

**有维修能力租车点的损坏汽车** <br />
在 $t$ 日开始时有维修能力租车点 $d$ 可用的损坏汽车数量。

\begin{equation}
\sum_{d2 \in \text{Depots}} 
\sum_{r \in \text{RentDays}} \text{pctDamaged}*\text{pctDepot}_{d2,d}*\text{pctRent}_{r}*\text{xRented}_{d2,(t-r)mod(6)}
\end{equation}

\begin{equation}
+ \sum_{d2 \in \text{Depots}} \text{xDtransfer}_{d2,d,(t-1)mod(6)} 
+ \text{xDleft}_{d,(t-1)mod(6)} = \text{xdamaged}_{d,t} 
\quad \forall d \in RD, t \in Days
\end{equation}

在 $t$ 日期间无维修能力租车点 $d$ 的未损坏汽车需求。

\begin{equation}
\text{xDamaged}_{d,t} = \text{xRepaired}_{d,t} +
\sum_{d2 \in \text{Depots} \cap RD} \text{xDtransfer}_{d,d2,t} + \text{xDleft}_{d,t}
\quad \forall d \in RD, t \in Days
\end{equation}

**维修能力** <br />
包括扩展能力选项在内的每个工作日 $t$ 的租车点 $d$ 的维修能力。

\begin{equation}
\text{xRepaired}_{Birmingham,t} \leq \text{capRepair}_{Birmingham}
+ 5*\text{yExpandCapB1} + 5*\text{yExpandCapB2}
\quad \forall t \in Days
\end{equation}

\begin{equation}
\text{xRepaired}_{Manchester,t} \leq \text{capRepair}_{Manchester}
+ 5*\text{yExpandCapM1} + 5*\text{yExpandCapM2}
\quad \forall t \in Days
\end{equation}

\begin{equation}
\text{xRepaired}_{Plymouth,t} \leq \text{capRepair}_{Plymouth}
+ 5*\text{yExpandCapP}
\quad \forall t \in Days
\end{equation}

**租车点需求** <br />
每个工作日 $t$ 的租车点 $d$ 的需求。

\begin{equation}
\text{xRented}_{d,t} \leq \text{demand}_{d,t}
\quad \forall d \in Depots, t \in Days
\end{equation}

**汽车数量** <br />
拥有的汽车总数等于周一从所有租车点租出3天的汽车数量,加上周二租出2天或3天的汽车数量,再加上周三早上在租车点的所有损坏和未损坏汽车数量。
原理: 选择一天(周三),计算在租车点归还的未损坏和损坏汽车数量,这些汽车在周三早上可用。计算已租出但未归还的汽车数量: 周一租出3天的汽车,周二租出2天或3天的汽车。

\begin{equation}
\sum_{d \in \text{Depots}} (0.25*\text{xRented}_{d,0} + 0.45*\text{xRented}_{d,1} + \text{xUndamaged}_{d,2}  + \text{xdamaged}_{d,2} ) = \text{xOwned}
\end{equation}

**维修能力扩展约束** <br />
只有在第一次扩展也执行的情况下,才能进行租车点的进一步扩展。

\begin{equation}
\text{yExpandCapB1} \geq \text{yExpandCapB2}
\end{equation}

\begin{equation}
\text{yExpandCapM1} \geq \text{yExpandCapM2}
\end{equation}

**最多扩展次数** <br />
最多只能执行三个选项。

\begin{equation}
\text{yExpandCapB1} + \text{yExpandCapB2} +
\text{yExpandCapM1} + 
\end{equation}

\begin{equation}
\text{yExpandCapM2} +
\text{yExpandCapP} \leq 3
\end{equation}


---
## Python 实现

我们导入 Gurobi Python 模块和其他 Python 库。

In [ ]:
# %pip install gurobipy

In [ ]:
import pandas as pd
from itertools import product

import gurobipy as gp
from gurobipy import GRB

# 测试环境: Python 3.11 & Gurobi 11.0

## 输入数据

我们定义模型的所有输入数据。

In [ ]:
# 一周中的租车点和工作日列表

depots = ['Glasgow','Manchester','Birmingham','Plymouth']
NRD = ['Glasgow'] # 无维修能力的租车点
RD =['Manchester','Birmingham','Plymouth'] # 有维修能力的租车点

days = [0,1,2,3,4,5] # 周一 = 0, 周二 = 1, ... 周六 = 5
rentDays = [1,2,3]

d2w, demand = gp.multidict({
    ('Glasgow',0): 100,
    ('Glasgow',1): 150,
    ('Glasgow',2): 135,
    ('Glasgow',3): 83,
    ('Glasgow',4): 120,
    ('Glasgow',5): 230,
    ('Manchester',0): 250,
    ('Manchester',1): 143,
    ('Manchester',2): 80,
    ('Manchester',3): 225,
    ('Manchester',4): 210,
    ('Manchester',5): 98,
    ('Birmingham',0): 95,
    ('Birmingham',1): 195,
    ('Birmingham',2): 242,
    ('Birmingham',3): 111,
    ('Birmingham',4): 70,
    ('Birmingham',5): 124,
    ('Plymouth',0): 160,
    ('Plymouth',1): 99,
    ('Plymouth',2): 55,
    ('Plymouth',3): 96,
    ('Plymouth',4): 115,
    ('Plymouth',5): 80
})

# 维修能力
depots, capacity = gp.multidict({
    ('Glasgow'): 0,
    ('Manchester'): 12,
    ('Birmingham'): 20,
    ('Plymouth'): 0
})

# 创建字典以记录
# pctRent: r天租期的汽车百分比
# cstMarginal: 租车r天的边际成本
# prcSameD: 租车r天并归还到同一租车点的价格
# prcOtherD: 租车r天并归还到其他租车点的价格
rentDays, pctRent, costMarginal, priceSameD, priceOtherD = gp.multidict({
    (1): [0.55,20,50,70],
    (2): [0.20,25,70,100],
    (3): [0.25,30,120,150]
})

# 每周拥有一辆汽车的成本
cstOwn = 15

# 损坏汽车费用
damagedFee = 10

# 创建字典以记录从租车点d租出并归还到租车点d2的汽车比例
d2d, pctFromToD = gp.multidict({
    ('Glasgow','Glasgow'): 0.6,
    ('Glasgow','Manchester'): 0.2,
    ('Glasgow','Birmingham'): 0.1,
    ('Glasgow','Plymouth'): 0.1,
    ('Manchester','Glasgow'): 0.15,
    ('Manchester','Manchester'): 0.55,
    ('Manchester','Birmingham'): 0.25,
    ('Manchester','Plymouth'): 0.05,
    ('Birmingham','Glasgow'): 0.15,
    ('Birmingham','Manchester'): 0.2,
    ('Birmingham','Birmingham'): 0.54,
    ('Birmingham','Plymouth'): 0.11,
    ('Plymouth','Glasgow'): 0.08,
    ('Plymouth','Manchester'): 0.12,
    ('Plymouth','Birmingham'): 0.27,
    ('Plymouth','Plymouth'): 0.53
})

# 创建字典以记录汽车的转移成本
d2d, cstFromToD = gp.multidict({
    ('Glasgow','Glasgow'): 0.001,
    ('Glasgow','Manchester'): 20,
    ('Glasgow','Birmingham'): 30,
    ('Glasgow','Plymouth'): 50,
    ('Manchester','Glasgow'): 20,
    ('Manchester','Manchester'): 0.001,
    ('Manchester','Birmingham'): 15,
    ('Manchester','Plymouth'): 35,
    ('Birmingham','Glasgow'): 30,
    ('Birmingham','Manchester'): 15,
    ('Birmingham','Birmingham'): 0.001,
    ('Birmingham','Plymouth'): 25,
    ('Plymouth','Glasgow'): 50,
    ('Plymouth','Manchester'): 35,
    ('Plymouth','Birmingham'): 25,
    ('Plymouth','Plymouth'): 0.001
})

# 未损坏和损坏汽车归还的比例
pctUndamaged = 0.9
pctDamaged = 0.1

# 维修能力扩展的二元成本
cstExpCapB1 = 18000
cstExpCapB2 = 8000
cstExpCapM1 = 20000
cstExpCapM2 = 5000
cstExpCapP = 19000


### 预处理
我们准备数据结构以构建混合整数线性规划模型。

In [ ]:
# 构建元组列表 (租车点, 租车点2), 其中 d != d2

list_d2notd = []

for d,d2 in d2d:
    if (d != d2):
        tp = d,d2
        list_d2notd.append(tp)

d2notd = gp.tuplelist(list_d2notd)

# 构建元组列表 (租车点, 租车点2, 工作日)
list_dd2t = []

for d,d2 in d2notd:
    for t in days:
        tp = d,d2,t 
        list_dd2t.append(tp)
                    
dd2t = gp.tuplelist(list_dd2t)

# 构建元组列表 (租车点, 租期)
list_dr = []

for d in depots:
    for r in rentDays:
        tp = d,r
        list_dr.append(tp)
        
dr = gp.tuplelist(list_dr) 

# 构建元组列表 (租车点, 工作日, 租期 )
list_dtr = []

for d in depots:
    for t in days:
            for r in rentDays:
                tp = d,t,r
                list_dtr.append(tp)
                
dtr = gp.tuplelist(list_dtr) 

# 构建元组列表 (租车点, 租车点2, 工作日, 租期)
list_dd2tr = []

for d,d2 in d2notd:
    for t in days:
        for r in rentDays:
            tp = d,d2,t,r
            list_dd2tr.append(tp)
                    
                    
dd2tr = gp.tuplelist(list_dd2tr)

## 模型部署
我们创建一个模型和变量。主要决策变量是选择在哪些租车点扩展维修能力,拥有多少辆车,以及每周工作日开始时应该将它们放在哪个租车点,以最大化每周利润。

In [ ]:
model = gp.Model('RentalCar1')

# 拥有的汽车数量
n = model.addVar(name="cars")

# 未损坏汽车数量
nu = model.addVars(d2w, name="UDcars")

# 损坏汽车数量
nd = model.addVars(d2w, name="Dcars")

# 租出(已租)的汽车数量不能超过需求
tr = model.addVars(d2w, ub=demand, name="Hcars")
#for d,t in d2w:
    #tr[d,t].lb = 1

# 未损坏汽车的期末库存
eu = model.addVars(d2w, name="EUDcars")

# 损坏汽车的期末库存
ed = model.addVars(d2w, name="EDcars")

# 转移的未损坏汽车数量
tu = model.addVars(dd2t, name="TUDcars")

# 转移的损坏汽车数量
td = model.addVars(dd2t, name="TDcars")

# 维修的损坏汽车数量
rp = model.addVars(d2w, name="RPcars")

# 维修的损坏汽车数量不能超过租车点维修能力
for d,t in d2w:
    if d == 'Glasgow':
        rp[d,t].ub = capacity[d] #格拉斯哥的维修能力为零

# 维修能力扩展的二元变量
zB1 = model.addVar(vtype=GRB.BINARY, name='expCapB1') # 伯明翰的扩展

zB2 = model.addVar(vtype=GRB.BINARY, name='expCapB2') # 伯明翰的进一步扩展

zM1 = model.addVar(vtype=GRB.BINARY, name='expCapM1') # 曼彻斯特的扩展

zM2 = model.addVar(vtype=GRB.BINARY, name='expCapM2') # 曼彻斯特的进一步扩展

zP = model.addVar(vtype=GRB.BINARY, name='expCapP') # 普利茅斯的扩展

Using license file c:\gurobi\gurobi.lic


### 约束条件
在 $t$ 日开始时无维修能力租车点 $d$ 可用的未损坏汽车数量应等于在 $t$ 日期间无维修能力租车点 $d$ 的未损坏汽车需求。

In [ ]:
# 未损坏汽车进入无维修能力租车点的约束条件(平衡方程的左侧 - 可用性)

UDcarsNRD_L = model.addConstrs((gp.quicksum(pctUndamaged*pctFromToD[d2,d]*pctRent[r]*tr[d2,(t-r)%6 ] for d2,r in dr ) 
                              + gp.quicksum(tu.select('*',d,(t-1)%6)  ) 
                              + eu[d,(t-1)%6 ] == nu[d,t] for d in NRD for t in days ), 
                             name="UDcarsNRD_L")

# 未损坏汽车离开无维修能力租车点的约束条件(平衡方程的右侧 - 需求)

UDcarsNRD_R = model.addConstrs((tr[d,t] 
                                + gp.quicksum(tu.select(d,'*',t )) 
                                + eu[d,t] == nu[d,t] for d in NRD for t in days ), name='UDcarsNRD_R' )

在 $t$ 日开始时有维修能力租车点 $d$ 可用的未损坏汽车数量应等于在 $t$ 日期间有维修能力租车点 $d$ 的未损坏汽车需求。

In [ ]:
# 未损坏汽车进入有维修能力租车点的约束条件(平衡方程的左侧 - 可用性)

UDcarsRD_L = model.addConstrs((gp.quicksum(pctUndamaged*pctFromToD[d2,d]*pctRent[r]*tr[d2,(t-r)%6 ] for d2,r in dr ) 
                              + gp.quicksum(tu.select('*',d,(t-1)%6)  ) + rp[d, (t-1)%6 ]
                              + eu[d,(t-1)%6 ] == nu[d,t] for d in RD for t in days ), 
                             name="UDcarsRD_L")

# 未损坏汽车离开有维修能力租车点的约束条件(平衡方程的右侧 - 需求)

UDcarsRD_R = model.addConstrs((tr[d,t] 
                                + gp.quicksum(tu.select(d,'*',t ) ) 
                                + eu[d,t] == nu[d,t] for d in RD for t in days ), name='UDcarsRD_R' )

在 $t$ 日开始时无维修能力租车点 $d$ 可用的损坏汽车数量应等于在 $t$ 日期间无维修能力租车点 $d$ 的未损坏汽车需求。

In [ ]:
# 损坏汽车进入无维修能力租车点的约束条件(平衡方程的左侧 - 可用性)

DcarsNRD_L = model.addConstrs((gp.quicksum(pctDamaged*pctFromToD[d2,d]*pctRent[r]*tr[d2,(t-r)%6 ] for d2,r in dr ) 
                              + ed[d,(t-1)%6 ] == nd[d,t] for d in NRD for t in days ), 
                             name="DcarsNRD_L")

# 损坏汽车离开无维修能力租车点的约束条件(平衡方程的右侧 - 需求)

DcarsNRD_R = model.addConstrs(( gp.quicksum(td[d,d2,t] for d2 in RD ) 
                                + ed[d,t] == nd[d,t] for d in NRD for t in days ), name='DcarsNRD_R' )

在 $t$ 日开始时有维修能力租车点 $d$ 可用的损坏汽车数量应等于在 $t$ 日期间无维修能力租车点 $d$ 的未损坏汽车需求。

In [ ]:
# 损坏汽车进入有维修能力租车点的约束条件(平衡方程的左侧 - 可用性)

DcarsRD_L = model.addConstrs((gp.quicksum(pctDamaged*pctFromToD[d2,d]*pctRent[r]*tr[d2,(t-r)%6 ] for d2,r in dr )
                              + gp.quicksum(td[d2,d,(t-1)%6 ] for d2, dd in d2notd if (dd == d)) 
                              + ed[d,(t-1)%6 ] == nd[d,t] for d in RD for t in days ), 
                             name="DcarsRD_L")

# 损坏汽车离开有维修能力租车点的约束条件(平衡方程的右侧 - 需求)

DcarsND_R = model.addConstrs((rp[d,t] + gp.quicksum(td[d,d2,t ] for d2 in NRD ) 
                                + ed[d,t] == nd[d,t] for d in RD for t in days ), name='DcarsND_R' )

拥有的汽车总数等于周一从所有租车点租出3天的汽车数量,加上周二租出2天或3天的汽车数量,再加上周三早上在租车点的所有损坏和未损坏汽车数量。

In [ ]:
# 拥有的汽车总数约束条件
# 注意: 25%的汽车租期为3天, 20% + 25% = 45%的汽车租期为2天或3天

carsConstr = model.addConstr((gp.quicksum(0.25*tr[d,0] + 0.45*tr[d,1] + nu[d,2] + nd[d,2] for d in depots ) 
                              == n ),name='carsConstr')

维修能力扩展约束。

In [ ]:
# 租车点维修能力约束条件

RepairCapB = model.addConstrs((rp['Birmingham',t] <= capacity['Birmingham'] 
                               + 5*(zB1 + zB2) for t in days ), name='RepairCapB')

RepairCapM = model.addConstrs((rp['Manchester',t] <= capacity['Manchester'] 
                               + 5*(zM1 + zM2) for t in days ), name='RepairCapM')

RepairCapP = model.addConstrs((rp['Plymouth',t] <= capacity['Plymouth'] 
                               + 5*zP for t in days ), name='RepairCapP')

# 进一步扩展约束条件

expandB1B2 = model.addConstr((zB1 >= zB2 ), name='expandB1B2')

expandM1M2 = model.addConstr((zM1 >= zM2 ), name='expandM1M2')

# 最多只能执行三个扩展选项

atMost3 = model.addConstr((zB1 + zB2 + zM1 + zM2 + zP <= 3), name='atMost3')

目标函数是最大化利润。

In [ ]:
# 最大化利润目标函数。

model.setObjective((
    gp.quicksum(pctFromToD[d,d]*pctRent[r]*(priceSameD[r] - costMarginal[r] + damagedFee)*tr[d,t] for d,t,r in dtr )
    + gp.quicksum(pctFromToD[d,d2]*pctRent[r]*(priceOtherD[r]-costMarginal[r]+damagedFee)*tr[d,t] for d,d2,t,r in dd2tr)
    - gp.quicksum(cstFromToD[d,d2]*tu[d,d2,t] for d,d2,t in dd2t) 
    - gp.quicksum(cstFromToD[d,d2]*td[d,d2,t] for d,d2,t in dd2t) 
    -(cstExpCapB1*zB1 + cstExpCapB2*zB2 + cstExpCapM1*zM1 + cstExpCapM2*zM2 + cstExpCapP*zP) - cstOwn*n ), GRB.MAXIMIZE)

In [ ]:
# 验证模型公式

model.write('CarRental2.lp')

# 运行优化引擎

model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 118 rows, 294 columns and 1136 nonzeros
Model fingerprint: 0xe441c90b
Variable types: 289 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e-03, 5e+00]
  Objective range  [2e+01, 2e+04]
  Bounds range     [1e+00, 3e+02]
  RHS range        [3e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 49 rows and 91 columns
Presolve time: 0.00s
Presolved: 69 rows, 203 columns, 987 nonzeros
Variable types: 198 continuous, 5 integer (5 binary)

Root relaxation: objective 1.357349e+05, 58 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 135734.925    0    2   -0.00000 135734.925      -     -    0s
H    0     0                    130734.92534 135734.925  3.8

---
## 分析

In [ ]:
# 输出报告

# 拥有的汽车总数
print(f"最优的拥有汽车数量是: {round(n.x)}")

# 最优利润
print(f"最优利润是: {'${:,.2f}'.format(round(model.objVal,2))}")


The optimal number of cars to be owned is: 983
The optimal profit is: $132,341.47


In [ ]:
# 维修能力扩展计划

if zB1.x > 0.5:
    print(f"伯明翰租车点的维修能力扩展到每天多5辆车")
    if zB2.x > 0.5:
        print(f"伯明翰租车点的维修能力进一步扩展到每天多5辆车")

if zM1.x > 0.5:
    print(f"曼彻斯特租车点的维修能力扩展到每天多5辆车")
    if zM2.x > 0.5:
        print(f"曼彻斯特租车点的维修能力进一步扩展到每天多5辆车")

if zP.x > 0.5:
    print(f"普利茅斯租车点的维修能力扩展到每天多5辆车")

The depot at Manchester expanded its capacity to 5 more cars per day
The depot at Manchester further expanded 5 more cars per day of additional capacity
The depot at Plymouth expanded its capacity to 5 more cars per day


In [ ]:
# 创建一个列表以将每一天的数字标签转换为实际的日期名称
dayname = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']

# 每天开始时租车点的未损坏汽车数量。
print("\n\n_________________________________________________________________________________")
print(f"每天开始时租车点的未损坏汽车数量: ")
print("_________________________________________________________________________________")

undamaged_cars = pd.DataFrame(
    {
        "Day": [dayname[t] for t in days],
        "Glasgow": [round(nu['Glasgow',t].x) for t in days],
        "Manchester": [round(nu['Manchester',t].x) for t in days],
        "Birmingham": [round(nu['Birmingham',t].x) for t in days],
        "Plymouth": [round(nu['Plymouth',t].x) for t in days],
    }
)
undamaged_cars.index=[''] * len(undamaged_cars)
undamaged_cars



_________________________________________________________________________________
Estimated number of undamaged cars in depot at the beginning of each day: 
_________________________________________________________________________________


,Day,Glasgow,Manchester,Birmingham,Plymouth
,Monday,95,227,242,66
,Tuesday,99,162,282,65
,Wednesday,100,168,242,67
,Thursday,101,233,168,80
,Friday,117,226,162,71
,Saturday,103,182,232,67


In [ ]:
# 每天开始时租车点的损坏汽车数量。
print("_________________________________________________________________________________")
print(f"每天开始时租车点的损坏汽车数量: ")
print("_________________________________________________________________________________")

damaged_cars = pd.DataFrame(
    {
        "Day": [dayname[t] for t in days],
        "Glasgow": [round(nd['Glasgow',t].x) for t in days],
        "Manchester": [round(nd['Manchester',t].x) for t in days],
        "Birmingham": [round(nd['Birmingham',t].x) for t in days],
        "Plymouth": [round(nd['Plymouth',t].x) for t in days],
    }
)
damaged_cars.index=[''] * len(damaged_cars)
damaged_cars

_________________________________________________________________________________
Estimated number of damaged cars in depot at the beginning of each day: 
_________________________________________________________________________________


,Day,Glasgow,Manchester,Birmingham,Plymouth
,Monday,12,22,20,7
,Tuesday,11,24,20,8
,Wednesday,11,22,20,7
,Thursday,11,25,24,9
,Friday,19,22,20,11
,Saturday,17,22,20,13


In [ ]:
# 每天从每个租车点租出的未损坏汽车数量。
print("_________________________________________________________________________________")
print(f"每天从每个租车点租出的未损坏汽车数量: ")
print("_________________________________________________________________________________")

rentedOut = {}

for d in depots:
    for t in days:
        count = 0
        for d2 in depots:
            for r in rentDays:
                #print(f"Depot {d}, day {t}: cars rented out {tr[d,t].x}")
                count += pctUndamaged*pctFromToD[d,d2]*pctRent[r]*tr[d,t].x
        rentedOut[d,t] = round(count)
    

#print(rentedOut)

rentout_cars = pd.DataFrame(
    {
        "Day": [dayname[t] for t in days],
        "Glasgow": [round(rentedOut['Glasgow',t]) for t in days],
        "Manchester": [round(rentedOut['Manchester',t]) for t in days],
        "Birmingham": [round(rentedOut['Birmingham',t]) for t in days],
        "Plymouth": [round(rentedOut['Plymouth',t]) for t in days],
    }
)
rentout_cars.index=[''] * len(rentout_cars)
rentout_cars

_________________________________________________________________________________
Estimated number of undamaged cars rented out from each depot and day: 
_________________________________________________________________________________


,Day,Glasgow,Manchester,Birmingham,Plymouth
,Monday,85,204,86,59
,Tuesday,89,129,176,58
,Wednesday,90,72,218,50
,Thursday,75,203,100,72
,Friday,106,189,63,64
,Saturday,93,88,112,61


---
## 参考文献

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Copyright © 2020 Gurobi Optimization, LLC